In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import plotly.express as px
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import requests
import re


In [3]:

#Data Scraping Functions

def catch_links_all_pages(url):
    oglasi = []
    page1 = requests.get(url)
    soup1 = BeautifulSoup(page1.content, "html.parser")

    #Extract the number of pages from navigation bar
    page_nav_class = soup1.find_all('ul', attrs={'class':"pagination"}) 
    page_nav = page_nav_class[0].find_all('li')
    #Take the last navigation element
    last = page_nav[-1]
    #Extract the number and clean data
    page_number = last.find('a').get('href')[-3:]
    page_number = re.sub("[^0-9]", "", page_number)
    fix = "&num="
    
    #Create a list of all pages with links
    pages = []
    pages.append(url)
    for i in range(2,int(page_number)+1):
            link = url + fix + str(i)
            pages.append(link)
    
    pages = list(set(pages))
       
    #Scrape all resluts from every page
    for page in pages:        
        web_page = requests.get(page)
        soup1 = BeautifulSoup(web_page.content, "html.parser")
        results = soup1.find_all('div', attrs={'class':"OglasiRezHolder"}) 
        for result in results:
            try:
                link = result.find('a').get('href')
                oglasi.append(link)
            except:
                pass

    oglasi = list(set(oglasi))

    return oglasi

def data_dict_index(url):
    page = requests.get(url)    

    soup = BeautifulSoup(page.content, "html.parser")

    my_dict = {}
    osnovno = soup.find_all('li', attrs={'class':'labela'})
    for row in osnovno:
        a = row.text
        b = row.findNextSibling().get_text().replace("\r\n", "")

        if a in my_dict.keys():
            if b < my_dict[a]:
                my_dict[a] = b
            else:
                pass
        else:
            my_dict[a] = b
    
    my_dict["Cijena"] = soup.find('div', attrs={'class':'price'}).find('span').text
    my_dict["Link"] = url

    datum = soup.find_all('div', attrs={'class':'published'})
    regex = r"\d{2}.\d{2}.\d{4}"
    date = re.findall(regex,datum[1].get_text())[0]
    my_dict["Datum"] = date
    return my_dict

def dataframe_cleaner(df):
    #df = pd.DataFrame(data_list)

    #Data Cleaning

    #Drop rows w/o sqm, convert sqm to float
    df = df[df['Stambena površina u m2'].notna()]
    df["Stambena površina u m2"] = df["Stambena površina u m2"].str.replace(",",".")
    df["Stambena površina u m2"] = df["Stambena površina u m2"].astype('float')

    #Convert price to float, remove no price rows, rempve unrealistic low prices
    df["Cijena"] = df["Cijena"].str.replace("€","")

    df["Cijena"] = df["Cijena"].str.replace(".","")
    df["Cijena"] = df["Cijena"].str.replace(",",".")

    df["Cijena"] = df["Cijena"].str.replace(" ","")

    df = df[df["Cijena"] != "0,00 "]
    df["Cijena"] = df["Cijena"].astype('float')
    df = df[df["Cijena"] > 10000]

    #Replace Nan with "No info"
    df['Godina izgradnje'].fillna('No INFO', inplace=True)

    #Remove where m2 == 0
    df = df[df["Stambena površina u m2"] > 10]
    df = df[df["Stambena površina u m2"] < 10000]




    #Add a price per sqm meter column
    df["€/m²"] = round(df["Cijena"] / df["Stambena površina u m2"],2)

    #Convert Datum to datetime
    df["Datum"] = pd.to_datetime(df["Datum"], format="%d.%m.%Y")

    #Calculate days online
    df['Days Online'] = ( pd.Timestamp('now') - df['Datum']).dt.days

    return df

In [17]:
url = "https://www.index.hr/oglasi/prodaja-stanova/gid/3278?pojam=&sortby=1&elementsNum=100&cijenaod=0&cijenado=21000000&tipoglasa=1&pojamZup=1154&grad=1294&naselje=&attr_Int_988=&attr_Int_887=&attr_bit_stan=&attr_bit_brojEtaza=&attr_gr_93_1=&attr_gr_93_2=&attr_Int_978=&attr_Int_1334=&attr_bit_eneregetskiCertifikat=&vezani_na=988-887_562-563_978-1334"
oglasi = []
page1 = requests.get(url)
soup1 = BeautifulSoup(page1.content, "html.parser")

#Extract the number of pages from navigation bar
page_nav_class = soup1.find_all('ul', attrs={'class':"pagination"}) 
page_nav = page_nav_class[0].find_all('li')


pages = []

if not page_nav:
    pages.append(url)

else:
    #Take the last navigation element
    last = page_nav[-1]
    #Extract the number and clean data
    page_number = last.find('a').get('href')[-3:]
    page_number = re.sub("[^0-9]", "", page_number)
    fix = "&num="
    # print(page_number)

    #Create a list of all pages with links
    pages.append(url)
    for i in range(2,int(page_number)+1):
            link = url + fix + str(i)
            pages.append(link)

    pages = list(set(pages))

#Scrape all resluts from every page
for page in pages:        
    web_page = requests.get(page)
    soup1 = BeautifulSoup(web_page.content, "html.parser")
    results = soup1.find_all('div', attrs={'class':"OglasiRezHolder"}) 
    # print(len(results))
    for result in results:
        try:
            link = result.find('a').get('href')
            oglasi.append(link)
        except:
            pass

print(len(oglasi))

589


In [12]:

url = "https://www.index.hr/oglasi/najam-stanova/gid/3279?pojam=&sortby=1&elementsNum=100&cijenaod=0&cijenado=21000000&tipoglasa=1&pojamZup=1166&grad=0&naselje=&attr_Int_988=&attr_Int_887=&attr_bit_stan=&attr_bit_brojEtaza=&attr_gr_93_1=&attr_gr_93_2=&attr_Int_978=&attr_Int_1334=&attr_bit_eneregetskiCertifikat=&vezani_na=988-887_562-563_978-1334"

city_name = "Varazdin"
ime = "_varazdin"



#find all pages
pages = catch_links_all_pages(url)
len(pages)

IndexError: list index out of range

In [4]:
url = 'https://www.index.hr/oglasi/najam-stanova/gid/3279?pojam=&sortby=1&elementsNum=100&cijenaod=0&cijenado=21000000&tipoglasa=1&pojamZup=1162&grad=1511&naselje=&attr_Int_988=&attr_Int_887=&attr_bit_stan=&attr_bit_brojEtaza=&attr_gr_93_1=&attr_gr_93_2=&attr_Int_978=&attr_Int_1334=&attr_bit_eneregetskiCertifikat=&vezani_na=988-887_562-563_978-1334'

url_ro = "https://www.index.hr/oglasi/prodaja-stanova/gid/3278?pojam=&sortby=1&elementsNum=100&cijenaod=0&cijenado=21000000&tipoglasa=1&pojamZup=1154&grad=1294&naselje=&attr_Int_988=&attr_Int_887=&attr_bit_stan=&attr_bit_brojEtaza=&attr_gr_93_1=&attr_gr_93_2=&attr_Int_978=&attr_Int_1334=&attr_bit_eneregetskiCertifikat=&vezani_na=988-887_562-563_978-1334"
pages = catch_links_all_pages(url_ro)




#scrape all pages
data_list = []
for page in pages:
    try:
        data_list.append(data_dict_index(page))
    except:
        pass

#create dataframe
df = pd.DataFrame(data_list)

In [18]:
len(df_raw.index)

2692

In [24]:
df_raw

,Županija,Grad/općina,Naselje,Stambena površina u m2,Broj soba,Kat,Zamjena,Tip oglasa,Prodavač,Broj etaža stana,...,Površina balkona u m2,Grijanje,email,Tip kuće,Broj etaža kuće,Površina okućnice u m2,Tip nekretnine za odmor,Provizija,Pozicija poslovnog prostora,Namjena poslovnog prostora
0,Primorsko-goranska,Rijeka,Podmurvice,72,3-3.5 sobni\n,3\n,ne,ponuda,pravna osoba,Jednoetažni\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Primorsko-goranska,Rijeka,Centar,48,2,NaN,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Primorsko-goranska,Rijeka,Pećine,42,Garsonijera\n,NaN,ne,ponuda,pravna osoba,Jednoetažni\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Primorsko-goranska,Rijeka,Centar,100,4+\n,NaN,ne,ponuda,privatna osoba,Jednoetažni\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Primorsko-goranska,Rijeka,Zamet,70,4+\n,NaN,ne,ponuda,privatna osoba,Jednoetažni\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,Primorsko-goranska,Rijeka,Zamet,82,4+\n,NaN,ne,ponuda,privatna osoba,Jednoetažni\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2688,Primorsko-goranska,Rijeka,Pećine,50,1-1.5 sobni\n,NaN,ne,ponuda,pravna osoba,Jednoetažni\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2689,Primorsko-goranska,Rijeka,Centar,296,4+\n,NaN,ne,ponuda,pravna osoba,Jednoetažni\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2690,Primorsko-goranska,Rijeka,Belveder,60,2-2.5 sobni\n,NaN,ne,ponuda,pravna osoba,Jednoetažni\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_raw.to_excel("RENT PRICES RIJEKA.xlsx")


In [30]:
#CLEANING TEST

df_test = df_raw

print(len(df_test.index))

df_test = df_test[df_test['Stambena površina u m2'].notna()]

print(len(df_test.index))



2692
1854


In [16]:
pd.options.mode.chained_assignment = None 

df = df[df['Stambena površina u m2'].notna()]
df["Stambena površina u m2"] = df["Stambena površina u m2"].str.replace(",",".")
df["Stambena površina u m2"] = df["Stambena površina u m2"].astype('float')

#Convert price to float, remove no price rows, rempve unrealistic low prices
df["Cijena"] = df["Cijena"].str.replace("€","")

df["Cijena"] = df["Cijena"].str.replace(".","")
df["Cijena"] = df["Cijena"].str.replace(",",".")

df["Cijena"] = df["Cijena"].str.replace(" ","")

df = df[df["Cijena"] != "0,00 "]
df["Cijena"] = df["Cijena"].astype('float')

AttributeError: Can only use .str accessor with string values!

In [26]:
df.to_excel("RENT PRICES RIJEKA_CLEAN.xlsx")


In [19]:
len(df.index)

1854

In [4]:
df = pd.read_excel("media/2022-07-27_zagreb.xlsx")

df["Stambena površina u m2"].mean()
df["Stambena površina u m2"].mode()

# dj_avg_size = round(df["Stambena površina u m2"].mean(),2)



0    63.0
1    80.0
Name: Stambena površina u m2, dtype: float64

In [12]:
df["Stambena površina u m2"].value_counts(bins = 100).reset_index()
fig = px.bar(df["Stambena površina u m2"].value_counts(bins = 100).reset_index(), x="index", y="Stambena površina u m2")
fig

TypeError: Object of type Interval is not JSON serializable

TypeError: Object of type Interval is not JSON serializable

TypeError: Object of type Interval is not JSON serializable

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'index=%{x}<br>Stambena površina u m2=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([Interval(53.55, 62.06, closed='right'),
                          Interval(62.06, 70.57, closed='right'),
                          Interval(70.57, 79.08, closed='right'),
                          Interval(79.08, 87.59, closed='right'),
                          Interval(45.04, 53.55, closed='right'),
                          Interval(87.59, 96.1, closed='right'),
                          Interval(96.1, 104.61, closed='right'),
                          Interval(36.53, 45.04, closed='right'),
                          Interva

In [4]:
fig = px.histogram(df, x="Stambena površina u m2")
fig

In [9]:
fig = px.histogram(df, x="Godina izgradnje")
fig

In [10]:
fig = px.histogram(df, x="Cijena")
fig

In [2]:
df = pd.read_excel("Django_Objects.xlsx")
df

,Unnamed: 0,city,calendar_week,raw_entries,clean_entries,avg_price_sqrm,avg_size,avg_year
0,0,Rovinj,30,544,337,4279,91,1900
1,1,Zagreb,30,5836,4122,2616,92,2022
2,2,Varazdin,31,100,54,1735,66,1968
3,3,Varazdin,32,100,56,1740,67,1968
4,4,Zagreb,32,5966,4138,2635,92,2022
5,5,Varazdin,32,100,53,1737,67,1968
6,6,Rijeka,33,1779,1319,1996,92,2020
7,7,Rovinj,33,569,337,4240,94,1900
8,8,Split,33,656,570,3616,85,2000
9,9,Varazdin,33,100,51,1813,77,2023


In [ ]:
fig = px.line(df, x="calendar_week", y="avg_price_sqrm", color = 'city', markers=True, height=600)
fig

In [5]:
fig = px.line(df, x="calendar_week", y="avg_size", color = 'city', markers=True, height=600)
fig


In [3]:
#PLOTLY FOR DJANGO OBJECTS
df = pd.read_excel("Django_Objects.xlsx")

for grad in list(df["city"].unique()):
    fig = px.line(df[df["city"] == grad], x="calendar_week", y="avg_price_sqrm", color = 'city', markers=True, title=grad)
    fig.show()

In [4]:
for grad in list(df["city"].unique()):
    fig = px.line(df[df["city"] == grad], x="calendar_week", y="raw_entries", color = 'city', markers=True, title=grad)
    fig.show()

In [6]:
for grad in list(df["city"].unique()):
    fig = px.line(df[df["city"] == grad], x="calendar_week", y="clean_entries", color = 'city', markers=True, title=grad)
    fig.show()

In [20]:
#PLOTLY FOR DJANGO OBJECTS
df = pd.read_excel("Django_Objects.xlsx")

fig = px.line(df, x="calendar_week", y="avg_price_sqrm", color = 'city', markers=True, title=grad, width=1000, height=600)
fig.show()

In [19]:
import os
cwd = os.getcwd()
onlyfiles = [f for f in listdir(cwd) ]
onlyfiles


['dashboard',
 'db.sqlite3',
 'manage.py',
 'media',
 'raw',
 'RealEstateApp',
 'RealEstateApp.code-workspace',
 'static']

In [4]:
cwd = os.getcwd()

path = cwd + "\\raw"
onlyfiles = [f for f in listdir(path)]

for file in onlyfiles:
    file_path = path + "\\" + file
    print(file_path)

c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-07-22_RAW_varazdin.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-07-27_RAW_rovinj.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-07-27_RAW_varazdin.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-07-27_RAW_zagreb.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-08-05_RAW_varazdin.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-08-07_RAW_rovinj.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-08-07_RAW_varazdin.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-08-08_RAW_rovi

In [20]:
path = cwd + "\\raw"
onlyfiles = [f for f in listdir(path) ]
onlyfiles

['2022-07-22_RAW_varazdin.xlsx',
 '2022-07-27_RAW_rovinj.xlsx',
 '2022-07-27_RAW_varazdin.xlsx',
 '2022-07-27_RAW_zagreb.xlsx',
 '2022-08-05_RAW_varazdin.xlsx',
 '2022-08-07_RAW_rovinj.xlsx',
 '2022-08-07_RAW_varazdin.xlsx',
 '2022-08-08_RAW_rovinj.xlsx',
 '2022-08-11_RAW_rovinj.xlsx',
 '2022-08-11_RAW_varazdin.xlsx',
 '2022-08-12_RAW_rovinj.xlsx',
 '2022-08-12_RAW_zagreb.xlsx',
 '2022-08-14_RAW_varazdin.xlsx',
 '2022-08-15_RAW_rijeka.xlsx',
 '2022-08-15_RAW_rovinj.xlsx',
 '2022-08-15_RAW_split.xlsx',
 '2022-08-19_RAW_varazdin.xlsx',
 '2022-08-19_RAW_zagreb.xlsx',
 '2022-08-24_RAW_rijeka.xlsx',
 '2022-08-24_RAW_rovinj.xlsx',
 '2022-08-24_RAW_split.xlsx',
 '2022-08-24_RAW_varazdin.xlsx',
 '2022-08-24_RAW_zagreb.xlsx',
 '2022-08-30_RAW_rijeka.xlsx',
 '2022-08-30_RAW_rovinj.xlsx',
 '2022-08-30_RAW_split.xlsx',
 '2022-08-30_RAW_varazdin.xlsx',
 '2022-08-30_RAW_zagreb.xlsx',
 '2022-09-04_RAW_rijeka.xlsx',
 '2022-09-04_RAW_rovinj.xlsx',
 '2022-09-04_RAW_split.xlsx',
 '2022-09-04_RAW_varazdin

In [22]:
file_path = path + "\\" + onlyfiles[0]
df = pd.read_excel(file_path)

In [23]:
df

,Unnamed: 0,Županija,Grad/općina,Naselje,Stambena površina u m2,Broj soba,Zamjena,Tip oglasa,Prodavač,Detaljnije o kući,...,Površina terase u m2,Namještenost,Površina vrta u m2,Ulaz,Grijanje,email,tel ili mobitel,Tip nekretnine za odmor,O zemljištu,Pozicija poslovnog prostora
0,0,Istarska,Rovinj,Rovinj,50,3-3.5 sobni4+\n,ne,ponuda,pravna osoba,Novogradnja\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Istarska,Rovinj,Rovinj,92,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Istarska,Rovinj,Rovinj,96,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Istarska,Rovinj,Rovinj,210,4+\n,ne,ponuda,pravna osoba,Novogradnja\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Istarska,Rovinj,Rovinj,32,3-3.5 sobni\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,539,Istarska,Rovinj,Rovinj,97,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,540,Istarska,Rovinj,Rovinj,80,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U zgradi
541,541,Istarska,Rovinj,Rovinj,60,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542,542,Istarska,Rovinj,Rovinj,91,2-2.5 sobni4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
name = onlyfiles[1].split("_")

In [38]:

from datetime import datetime
test = datetime.strptime(name[0], '%Y-%m-%d').date()
test, test.isocalendar()[1]

(datetime.date(2022, 7, 27), 30)

In [31]:
ime = "_" + name[2].split(".")[0]
city_name = name[2].split(".")[0].capitalize()


In [32]:
ime, city_name

('_rovinj', 'Rovinj')